In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [2]:
#load xml files and store in a list
xml_list=glob('./data_images/*.xml')
xml_list=list(map(lambda x: x.replace('\\','/'),xml_list))

In [3]:
xml_list

['./data_images/2007_000027.xml',
 './data_images/2007_000032.xml',
 './data_images/2007_000033.xml',
 './data_images/2007_000039.xml',
 './data_images/2007_000042.xml',
 './data_images/2007_000061.xml',
 './data_images/2007_000063.xml',
 './data_images/2007_000068.xml',
 './data_images/2007_000121.xml',
 './data_images/2007_000123.xml',
 './data_images/2007_000129.xml',
 './data_images/2007_000170.xml',
 './data_images/2007_000175.xml',
 './data_images/2007_000187.xml',
 './data_images/2007_000241.xml',
 './data_images/2007_000243.xml',
 './data_images/2007_000250.xml',
 './data_images/2007_000256.xml',
 './data_images/2007_000272.xml',
 './data_images/2007_000323.xml',
 './data_images/2007_000332.xml',
 './data_images/2007_000333.xml',
 './data_images/2007_000346.xml',
 './data_images/2007_000363.xml',
 './data_images/2007_000364.xml',
 './data_images/2007_000392.xml',
 './data_images/2007_000423.xml',
 './data_images/2007_000452.xml',
 './data_images/2007_000464.xml',
 './data_image

In [4]:
#extract file name
def extractText(filename):
    tree=et.parse(filename)
    root=tree.getroot()
    image_name=root.find('filename').text
    width=root.find('size').find('width').text
    height=root.find('size').find('height').text
    objs=root.findall('object')
    parser=[]
    for obj in objs:
        name=obj.find('name').text
        bndbox=obj.find('bndbox')
        xmin=bndbox.find('xmin').text
        xmax=bndbox.find('xmax').text
        ymin=bndbox.find('ymin').text
        ymax=bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
    return parser

In [5]:
parser_all=list(map(extractText,xml_list))

In [6]:
data=reduce(lambda x,y: x+y,parser_all)

In [7]:
df=pd.DataFrame(data,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])

In [8]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,2007_000027.jpg,486,500,person,174,349,101,351
1,2007_000032.jpg,500,281,aeroplane,104,375,78,183
2,2007_000032.jpg,500,281,aeroplane,133,197,88,123
3,2007_000032.jpg,500,281,person,195,213,180,229
4,2007_000032.jpg,500,281,person,26,44,189,238


In [9]:
df.shape

(22596, 8)

In [10]:
df['name'].value_counts()

name
person         7674
chair          1964
car            1708
bottle         1023
dog             939
bird            908
pottedplant     837
cat             750
boat            736
aeroplane       668
tvmonitor       636
horse           590
sheep           589
bicycle         575
motorbike       567
sofa            523
diningtable     500
cow             486
train           464
bus             459
Name: count, dtype: int64

# # Conversion
- centre_x=((xmin+xmax)/2)/width
- centre_y=((ymin+ymax)/2)/height
- w=(xmax-xmin)/width of the image
- h=(ymax-ymin)/width of the image


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22596 entries, 0 to 22595
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  22596 non-null  object
 1   width     22596 non-null  object
 2   height    22596 non-null  object
 3   name      22596 non-null  object
 4   xmin      22596 non-null  object
 5   xmax      22596 non-null  object
 6   ymin      22596 non-null  object
 7   ymax      22596 non-null  object
dtypes: object(8)
memory usage: 1.4+ MB


In [12]:
cols=['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(float)
df[cols] = df[cols].fillna(0)
df[cols]=df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22596 entries, 0 to 22595
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  22596 non-null  object
 1   width     22596 non-null  int32 
 2   height    22596 non-null  int32 
 3   name      22596 non-null  object
 4   xmin      22596 non-null  int32 
 5   xmax      22596 non-null  int32 
 6   ymin      22596 non-null  int32 
 7   ymax      22596 non-null  int32 
dtypes: int32(6), object(2)
memory usage: 882.8+ KB


In [13]:
df['centreX']=((df['xmin']+df['xmax'])/2)/df['width']
df['centreY']=((df['ymin']+df['ymax'])/2)/df['height']
df['w']=((df['xmax']-df['xmin']))/df['width']
df['h']=((df['ymax']-df['ymin']))/df['height']

In [14]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,centreX,centreY,w,h
0,2007_000027.jpg,486,500,person,174,349,101,351,0.538066,0.452000,0.360082,0.500000
1,2007_000032.jpg,500,281,aeroplane,104,375,78,183,0.479000,0.464413,0.542000,0.373665
2,2007_000032.jpg,500,281,aeroplane,133,197,88,123,0.330000,0.375445,0.128000,0.124555
3,2007_000032.jpg,500,281,person,195,213,180,229,0.408000,0.727758,0.036000,0.174377
4,2007_000032.jpg,500,281,person,26,44,189,238,0.070000,0.759786,0.036000,0.174377


In [15]:
images=df['filename'].unique()

In [16]:
len(images)

8280

In [17]:
img_df=pd.DataFrame(images,columns=['filename'])
img_train=tuple(img_df.sample(frac=0.8)['filename'])

In [18]:
img_test=tuple(img_df.query(f'filename not in {img_train}')['filename'])

In [19]:
len(img_train), len(img_test)

(6624, 1656)

In [20]:
train_df=df.query(f'filename in {img_train}')
test_df=df.query(f'filename in {img_test}')

In [21]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,centreX,centreY,w,h
0,2007_000027.jpg,486,500,person,174,349,101,351,0.538066,0.452000,0.360082,0.500000
1,2007_000032.jpg,500,281,aeroplane,104,375,78,183,0.479000,0.464413,0.542000,0.373665
2,2007_000032.jpg,500,281,aeroplane,133,197,88,123,0.330000,0.375445,0.128000,0.124555
3,2007_000032.jpg,500,281,person,195,213,180,229,0.408000,0.727758,0.036000,0.174377
4,2007_000032.jpg,500,281,person,26,44,189,238,0.070000,0.759786,0.036000,0.174377


In [22]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,centreX,centreY,w,h
18,2007_000123.jpg,500,375,train,1,358,26,340,0.359000,0.488000,0.714000,0.837333
60,2007_000464.jpg,375,500,cow,71,216,252,314,0.382667,0.566000,0.386667,0.124000
61,2007_000464.jpg,375,500,cow,58,241,202,295,0.398667,0.497000,0.488000,0.186000
66,2007_000491.jpg,500,334,cow,2,459,2,334,0.461000,0.502994,0.914000,0.994012
81,2007_000572.jpg,500,331,bicycle,1,235,178,331,0.236000,0.768882,0.468000,0.462236


## Define id number to object names

In [23]:
#label encodeing
def labelEncoding(x):
    labels={'person':0,'car':1,'chair':2,'bottle':3,'pottedplant':4,'bird':5,'dog':6,'sofa':7,'bicycle':8,'horse':9,'boat':10,'motorbike':11
           ,'cat':12,'tvmonitor':13,'cow':14,'sheep':15,'aeroplane':16,'train':17,'diningtable':18,'bus':19}
    return labels[x]

In [24]:
train_df['id']=train_df['name'].apply(labelEncoding)
test_df['id']=test_df['name'].apply(labelEncoding)

C:\Users\icyga\AppData\Local\Temp\ipykernel_10068\2341147973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id']=train_df['name'].apply(labelEncoding)
C:\Users\icyga\AppData\Local\Temp\ipykernel_10068\2341147973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id']=test_df['name'].apply(labelEncoding)


In [25]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,centreX,centreY,w,h,id
0,2007_000027.jpg,486,500,person,174,349,101,351,0.538066,0.452000,0.360082,0.500000,0
1,2007_000032.jpg,500,281,aeroplane,104,375,78,183,0.479000,0.464413,0.542000,0.373665,16
2,2007_000032.jpg,500,281,aeroplane,133,197,88,123,0.330000,0.375445,0.128000,0.124555,16
3,2007_000032.jpg,500,281,person,195,213,180,229,0.408000,0.727758,0.036000,0.174377,0
4,2007_000032.jpg,500,281,person,26,44,189,238,0.070000,0.759786,0.036000,0.174377,0


In [26]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,centreX,centreY,w,h,id
18,2007_000123.jpg,500,375,train,1,358,26,340,0.359000,0.488000,0.714000,0.837333,17
60,2007_000464.jpg,375,500,cow,71,216,252,314,0.382667,0.566000,0.386667,0.124000,14
61,2007_000464.jpg,375,500,cow,58,241,202,295,0.398667,0.497000,0.488000,0.186000,14
66,2007_000491.jpg,500,334,cow,2,459,2,334,0.461000,0.502994,0.914000,0.994012,14
81,2007_000572.jpg,500,331,bicycle,1,235,178,331,0.236000,0.768882,0.468000,0.462236,8


In [27]:
import os
from shutil import move

In [28]:
# Absolute paths for train and test folders
base_path = os.getcwd()
train_folder = os.path.join(base_path, 'data_images', 'train')
test_folder = os.path.join(base_path,  'data_images', 'test')
source_folder = os.path.join(base_path, 'data_images')
# Create directories if they do not exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

In [29]:
cols=['filename','id','centreX','centreY','w','h']
groupby_obj_train=train_df[cols].groupby('filename')
groupby_obj_test=test_df[cols].groupby('filename')

In [30]:
def saveData(filename, folderpath, groupObj):
    src = os.path.join(source_folder, filename)
    dst = os.path.join(folderpath, filename)
    
    # Debug statements to print paths
    print(f"Processing file: {filename}")
    print(f"Source: {src}")
    print(f"Destination: {dst}")
    
    # Check if the source file exists
    if os.path.exists(src):
        move(src, dst)
        # Save the labels
        textFileName = os.path.join(folderpath, os.path.splitext(filename)[0] + '.txt')
        groupObj.get_group(filename).set_index('filename').to_csv(textFileName, sep=' ', index=False, header=False)
    else:
        print(f"File not found: {src}")

In [31]:
fileNameSeries=pd.Series(groupby_obj_train.groups.keys())

In [32]:
fileNameSeries.head(2)

0    2007_000027.jpg
1    2007_000032.jpg
dtype: object

In [33]:
fileNameSeries.apply(saveData,args=(train_folder,groupby_obj_train))

Processing file: 2007_000027.jpg
Source: D:\yolo object detection\data_images\2007_000027.jpg
Destination: D:\yolo object detection\data_images\train\2007_000027.jpg
Processing file: 2007_000032.jpg
Source: D:\yolo object detection\data_images\2007_000032.jpg
Destination: D:\yolo object detection\data_images\train\2007_000032.jpg
Processing file: 2007_000033.jpg
Source: D:\yolo object detection\data_images\2007_000033.jpg
Destination: D:\yolo object detection\data_images\train\2007_000033.jpg
Processing file: 2007_000039.jpg
Source: D:\yolo object detection\data_images\2007_000039.jpg
Destination: D:\yolo object detection\data_images\train\2007_000039.jpg
Processing file: 2007_000042.jpg
Source: D:\yolo object detection\data_images\2007_000042.jpg
Destination: D:\yolo object detection\data_images\train\2007_000042.jpg
Processing file: 2007_000061.jpg
Source: D:\yolo object detection\data_images\2007_000061.jpg
Destination: D:\yolo object detection\data_images\train\2007_000061.jpg
Proc

0       None
1       None
2       None
3       None
4       None
        ... 
6619    None
6620    None
6621    None
6622    None
6623    None
Length: 6624, dtype: object

In [34]:
fileNameSeries

0       2007_000027.jpg
1       2007_000032.jpg
2       2007_000033.jpg
3       2007_000039.jpg
4       2007_000042.jpg
             ...       
6619    2010_000918.jpg
6620    2010_000920.jpg
6621    2010_000922.jpg
6622    2010_000923.jpg
6623    2010_000926.jpg
Length: 6624, dtype: object

In [35]:
fileNameSeriesTest=pd.Series(groupby_obj_test.groups.keys())

In [36]:
fileNameSeriesTest.apply(saveData,args=(test_folder,groupby_obj_test))

Processing file: 2007_000123.jpg
Source: D:\yolo object detection\data_images\2007_000123.jpg
Destination: D:\yolo object detection\data_images\test\2007_000123.jpg
Processing file: 2007_000464.jpg
Source: D:\yolo object detection\data_images\2007_000464.jpg
Destination: D:\yolo object detection\data_images\test\2007_000464.jpg
Processing file: 2007_000491.jpg
Source: D:\yolo object detection\data_images\2007_000491.jpg
Destination: D:\yolo object detection\data_images\test\2007_000491.jpg
Processing file: 2007_000572.jpg
Source: D:\yolo object detection\data_images\2007_000572.jpg
Destination: D:\yolo object detection\data_images\test\2007_000572.jpg
Processing file: 2007_000636.jpg
Source: D:\yolo object detection\data_images\2007_000636.jpg
Destination: D:\yolo object detection\data_images\test\2007_000636.jpg
Processing file: 2007_000762.jpg
Source: D:\yolo object detection\data_images\2007_000762.jpg
Destination: D:\yolo object detection\data_images\test\2007_000762.jpg
Processing

0       None
1       None
2       None
3       None
4       None
        ... 
1651    None
1652    None
1653    None
1654    None
1655    None
Length: 1656, dtype: object

In [37]:
fileNameSeriesTest

0       2007_000123.jpg
1       2007_000464.jpg
2       2007_000491.jpg
3       2007_000572.jpg
4       2007_000636.jpg
             ...       
1651    2010_000870.jpg
1652    2010_000874.jpg
1653    2010_000904.jpg
1654    2010_000908.jpg
1655    2010_000910.jpg
Length: 1656, dtype: object